# Eog gaze prediction###

This program is an extensions of Patrick's eye tracking Data Analysis. 

The goal is to figure out if a persons eye gaze position on the screen can be predicted by using linear regression and the recorded EOG data. 

Patrick's part of the program was translated to Pandas Data Frames for further use. The main and final Data Frame is called accepted_data_df, which contains the following information:

- Time stamp: Corresponding time
- ch0, ch1 and ch2: Eog data
- x and y: x and y coordinates corresponding to the point presented on the screen
- calibration point: Boolean
- stimulus: Boolean
- window: corresponding window number
- Peak1_media and Peak2_media: The Difference of Gaussians peak (DoG) time stamp median between ch0, ch1 and ch2
- ch0Peak1, ch1Peak1 and ch2Peak1: DoG for each channel
- ch0Peak2, ch1Peak2 and ch2Peak2: DoG second peak for each channel
- rejected: Boolean
- x_start and y_start: x and y coordinates at the start of a window
- x_end and y_end: x and y coordinates at the end of a window



# Setup and load data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.signal
import math
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.animation as animation


%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (16, 10)
filenames = 'data/run{}.csv'
datasets = []
for i in range(3):
    data = pd.read_csv(filenames.format(i), delimiter = ", ", comment = "#")
    datasets.append(data)

datasets_df = pd.concat(datasets)
datasets_df = datasets_df.reset_index(drop = True)

# Filters
 Data on Channel 1,2 and 3 are not damaged and are going to be used for the analysis.

In [ ]:
#Select useful Data
valid_datasets_df = datasets_df.loc[:, ('t','ch1','ch2','ch3','x','y','run')]
valid_datasets_df.columns = ['time stamp', 'ch0','ch1','ch2','x','y','run']

In [ ]:
#Median filter
filtered_datasets_df = valid_datasets_df.copy()
filtered_datasets_df[['ch0','ch1','ch2']] = sp.signal.medfilt(filtered_datasets_df[['ch0','ch1','ch2']], kernel_size=(3, 1)) 
plt.figure()
plt.plot(filtered_datasets_df[['ch0','ch1','ch2']].head(5000))
plt.show()

# Finding stimulus times and calibration points

A stimulus corresponds to the point on screen changing position. We need to get the indices of stimuli changes, then we can take a look at the EOG data in a window around stimuli to filter out bad ones.

In [ ]:
#Adds to dataframe information about calibration points
calib_df = pd.DataFrame()
calib_df['calib_x'] = filtered_datasets_df['x'].isin([50.0, 2510.0,1280.0])
calib_df['calib_y'] = filtered_datasets_df['y'].isin([50.0, 2510.0,1280.0])

filtered_datasets_df["calibration point"] = np.nan
filtered_datasets_df["calibration point"]= calib_df[['calib_x','calib_y']].applymap(lambda x: True if (x == True) else False)

filtered_datasets_df.head()

In [ ]:
#Adds to dataframe information about stimulus times
stimulus_df = filtered_datasets_df[['x','y']].copy()
stimulus_df['stimulus_x'] = stimulus_df['x'].shift() != stimulus_df['x']
stimulus_df['stimulus_y'] = stimulus_df['y'].shift() != stimulus_df['y']

#Finds stimulus points
filtered_datasets_df['stimulus'] = np.nan
filtered_datasets_df['stimulus']= stimulus_df[['stimulus_x','stimulus_y']].applymap(lambda x: True if (x == True) else False)
filtered_datasets_df['stimulus'].iloc[0] = False

#Stimulus index
stimulus_idxs = filtered_datasets_df[ filtered_datasets_df['stimulus'] == True].index

#Stimulus time stamp
stimulus_time_stamps = filtered_datasets_df[ filtered_datasets_df['stimulus'] == True]['time stamp']
filtered_datasets_df.loc[filtered_datasets_df['stimulus'] == True].head()

# Difference of Gaussians for transition detection
The difference of gaussians filter provides a good way to detect rapid changes in a signal. We try to use it to find the times where the signal switches after stimulus exposure.

In [ ]:
def get_dog_peaks(data_window,window_nr,rejected):
    
    size = 21
    s1 = 5
    s2 = 10
    kernel1 = sp.signal.gaussian(size, s1)
    kernel1 /= sum(kernel1)
    kernel2 = sp.signal.gaussian(size, s2)
    kernel2 /= sum(kernel2)
    data_window = data_window.reset_index()
  
    channels =  data_window[['ch0','ch1','ch2']]
  
    # smooth data
    smoothed_eog1 = np.apply_along_axis(lambda channel: sp.signal.convolve(channel, kernel1, mode='valid'), 0, channels)
    smoothed_eog2 = np.apply_along_axis(lambda channel: sp.signal.convolve(channel, kernel2, mode='valid'), 0, channels)

    smoothed_eog1 = pd.DataFrame(smoothed_eog1)
    smoothed_eog1.columns = ['ch0','ch1','ch2']
    smoothed_eog2 = pd.DataFrame(smoothed_eog2)
    smoothed_eog2.columns = ['ch0','ch1','ch2']
    
    # smoothing removed border points
    data_window = data_window[10:-10]
    data_window.reset_index(inplace = True, drop= True)
    
    # calculate Difference of Gaussians
    dogs = smoothed_eog1 - smoothed_eog2
 
    # calculate absolute DoG values
    abs_dogs = np.abs(dogs)
    data_window[['ch0 abs dogs','ch1 abs dogs','ch2 abs dogs']] = abs_dogs
    
    # Find peaks
    abs_dogs[['ch0 peak','ch1 peak','ch2 peak']] = abs_dogs[['ch0','ch1','ch2']].apply(lambda x:(x > x.shift())&(x.iloc[::-1] > x.iloc[::-1].shift()))
    data_window[['ch0 peak','ch1 peak','ch2 peak']] = abs_dogs[['ch0 peak','ch1 peak','ch2 peak']]
    data_window.set_index("time stamp", inplace = True)
    abs_dogs_peaks = data_window[['ch0 abs dogs','ch1 abs dogs','ch2 abs dogs','ch0 peak','ch1 peak','ch2 peak']]
    channels = ['ch0{}{}','ch1{}{}','ch2{}{}']
    
    first_peaks_time =[]
    second_peaks_time = []
    x_list = []
    y_list = []
    
    for chan in channels:
        max_peak = abs_dogs_peaks.loc[abs_dogs_peaks[chan.format(' peak','')] == True]
       
        #Gets the maximal peaks values for each channel
        max_peak_ch = max_peak[chan.format(' abs',' dogs')].nlargest(2)
        max_peak_ch = max_peak_ch.reset_index()
       
        if len(max_peak_ch) > 1:
            x = max_peak_ch.values[0][0],max_peak_ch.values[1][0]
            y = max_peak_ch.values[0][1],max_peak_ch.values[1][1]
            
            x_list.append(x)
            y_list.append(y)
            first_peaks_time.append(max_peak_ch.values[0][0])
            second_peaks_time.append(max_peak_ch.values[1][0])
            filtered_datasets_df[chan.format('Peak','1')].loc[filtered_datasets_df['time stamp'] == max_peak_ch.values[0][0]] = True
            filtered_datasets_df[chan.format('Peak','2')].loc[filtered_datasets_df['time stamp'] == max_peak_ch.values[1][0]] = True
            
        else:
            rejected = True
    
    #Rejects samples, where the minimal distance to the median is less than 20 miliseconds
    sort_fp_times = np.sort(first_peaks_time)
    sort_sp_times = np.sort(second_peaks_time)
   
    diffs_fp = np.absolute(sort_fp_times [:2] - sort_fp_times [1:]) * 1000
    diffs_sp = np.absolute(sort_sp_times [:2] - sort_sp_times [1:]) * 1000
    diffs_fp.sort()
    diffs_sp.sort()
 
    if  len(diffs_fp) > 0 and len(diffs_sp) > 0:
        if diffs_fp[0] > 20 or diffs_sp[0] > 20 :
            rejected = True
    #Reject samples where second Dog peak is found before the first Dog peak
    if (sort_sp_times[1]-sort_fp_times[1]) < 0.03:
        rejected = True    
        
    # If sample is not rejected, add peaks information to general dataframe and plot the median for first and second dog peaks
    
    if rejected == False:  
        plt.figure()
        for i, chan in enumerate(channels):
            #Enters information about first and second Dog peaks time media
            filtered_datasets_df['Peak1_media'].loc[filtered_datasets_df['time stamp'] == sort_fp_times[1]] = True
            filtered_datasets_df['Peak2_media'].loc[filtered_datasets_df['time stamp'] == sort_sp_times[1]] = True
            
            plt.plot(data_window[['ch0 abs dogs','ch1 abs dogs','ch2 abs dogs']])
            plt.scatter(x_list[i],y_list[i],color= 'r')
            plt.axvline(sort_fp_times[1], color= 'blue')
            plt.axvline(sort_sp_times[1], color= 'green')
            plt.title('Difference of Gaussians First and Second Peak, Window {}'.format(window_nr))
            plt.xlabel('time (seconds)')
            plt.ylabel('EOG data')
            plt.legend()
    
    plt.show()      
    return rejected, window_nr

# Stimulus analysis in Data Windows

At this point the data is organized in time windows, which start with a stimulus and end just before the next stimulus is presented. Difference of Gaussians is used to identify main signal switches. This step allows to reject samples that are too noisy or corrupted. Difference of Gaussians first and Second peaks are going to be used as features for linear regression.
This cell will run for a while (about 10 min)

In [ ]:
filtered_datasets_df['window'] = np.nan
filtered_datasets_df[['Peak1_media','Peak2_media','ch0Peak1','ch1Peak1','ch2Peak1','ch0Peak2','ch1Peak2','ch2Peak2','rejected']]= pd.DataFrame([[False, False,False, False, False,False, False, False, False]], index=filtered_datasets_df.index)
filtered_datasets_df[['x_start','y_start','x_end','y_end']]= pd.DataFrame([[False, False,False, False]], index=filtered_datasets_df.index)
count= 0
window_nr = 0
stim_idx_prev = 0
for i,stim_idx in enumerate(stimulus_idxs[0:-1]):
    rejected = False
    data_window = filtered_datasets_df.iloc[stim_idx:stimulus_idxs[i+1]].copy()
  
    if  stim_idx in data_window.index:
    
        filtered_datasets_df['window'].iloc[stim_idx:stimulus_idxs[i+1]] = window_nr
        stim_time_stamp = data_window['time stamp'].loc[stim_idx]
        data_window = data_window.set_index(['time stamp'])
        
        #Enter information about start and end coordinates for the window
        filtered_datasets_df['x_start'].iloc[stim_idx_prev] = True
        filtered_datasets_df['y_start'].iloc[stim_idx_prev] = True
        filtered_datasets_df['x_end'].iloc[stim_idx] = True
        filtered_datasets_df['y_end'].iloc[stim_idx] = True
        stim_idx_prev = stim_idx

    else:
        rejected = True
        pass  
    
    if len(data_window) > 270:
        rejected = True
        
    #Calculate DOG first and second peaks, enters iformation in Dataframe
    if rejected == False:
        rejected, window_nr = get_dog_peaks(data_window,window_nr,rejected)

    
    data_window = filtered_datasets_df.loc[filtered_datasets_df['window'] == window_nr]
    
    peak1_media_stamp = data_window['time stamp'].loc[data_window['Peak1_media'] == True].values
    peak2_media_stamp = data_window['time stamp'].loc[data_window['Peak2_media'] == True].values
    
    if len(peak2_media_stamp) > 1 or len(peak1_media_stamp) >1:
        rejected = True   
        
    if rejected == True:
        filtered_datasets_df['rejected'].loc[filtered_datasets_df['window'] == window_nr] = True
        window_nr += 1

    if rejected == False:
        plt.figure()
        plt.plot(data_window['time stamp'],data_window[['ch0','ch1','ch2']])
        plt.title('First and Second DoG Peaks to detect main signal switches in EOG Data')
        plt.xlabel('time (seconds)')
        plt.ylabel('EOG Data')
        plt.axvline(stim_time_stamp, color='r')
        plt.axvline(peak1_media_stamp, color='b')
        plt.axvline(peak2_media_stamp, color='g')
        plt.legend()
        window_nr += 1
        plt.show()
        
  
   

  

# Collect features for Linear Regression

In [ ]:
# Creates a new data frame will all data windows that were not rejected
accepted_datasets_df = filtered_datasets_df[filtered_datasets_df['rejected'] == False].copy()
#all x-end points 
x_start = accepted_datasets_df['x'].loc[accepted_datasets_df['x_start'] == True].reset_index(drop = True)
#all y-end points
y_start = accepted_datasets_df['y'].loc[accepted_datasets_df['y_start'] == True].reset_index(drop = True)
#all x-end points 
x_end = accepted_datasets_df['x'].loc[accepted_datasets_df['x_end'] == True].reset_index(drop = True)
#all y-end points
y_end = accepted_datasets_df['y'].loc[accepted_datasets_df['y_end'] == True].reset_index(drop = True)
#value for each channel at start point or stimulus point
stim_eog0 = accepted_datasets_df['ch0'].loc[accepted_datasets_df['stimulus'] == True].reset_index(drop = True)
stim_eog1 = accepted_datasets_df['ch1'].loc[accepted_datasets_df['stimulus'] == True].reset_index(drop = True)
stim_eog2 = accepted_datasets_df['ch2'].loc[accepted_datasets_df['stimulus'] == True].reset_index(drop = True)
#value for each channel at dog  2
dog_2_Ch0 = accepted_datasets_df['ch0'].loc[accepted_datasets_df['ch0Peak2'] == True].reset_index(drop = True)
dog_2_Ch1 = accepted_datasets_df['ch1'].loc[accepted_datasets_df['ch1Peak2'] == True].reset_index(drop = True)
dog_2_Ch2 = accepted_datasets_df['ch2'].loc[accepted_datasets_df['ch2Peak2'] == True].reset_index(drop = True)
#value for each channel at dog  1
dog_1_Ch0 = accepted_datasets_df['ch0'].loc[accepted_datasets_df['ch0Peak1'] == True].reset_index(drop = True)
dog_1_Ch1 = accepted_datasets_df['ch1'].loc[accepted_datasets_df['ch1Peak1'] == True].reset_index(drop = True)
dog_1_Ch2 = accepted_datasets_df['ch2'].loc[accepted_datasets_df['ch2Peak1'] == True].reset_index(drop = True)
#Values for first and secong Dog peaks at peak median
dog_1_Ch0_media = accepted_datasets_df['ch0'].loc[accepted_datasets_df['Peak1_media'] == True].reset_index(drop = True)
dog_1_Ch1_media = accepted_datasets_df['ch1'].loc[accepted_datasets_df['Peak1_media'] == True].reset_index(drop = True)
dog_1_Ch2_media = accepted_datasets_df['ch2'].loc[accepted_datasets_df['Peak1_media'] == True].reset_index(drop = True)
dog_2_Ch0_media = accepted_datasets_df['ch0'].loc[accepted_datasets_df['Peak2_media'] == True].reset_index(drop = True)
dog_2_Ch1_media = accepted_datasets_df['ch1'].loc[accepted_datasets_df['Peak2_media'] == True].reset_index(drop = True)
dog_2_Ch2_media = accepted_datasets_df['ch2'].loc[accepted_datasets_df['Peak2_media'] == True].reset_index(drop = True)
#Different combinations for doing linear regresion
linear_regression_df = pd.DataFrame(
    {'dog_1_Ch0': dog_1_Ch0_media,
     'dog_1_Ch1': dog_1_Ch1_media,
     'dog_1_Ch2': dog_1_Ch2_media,
     'dog_2_Ch0': dog_2_Ch0_media,
     'dog_2_Ch1': dog_2_Ch1_media,
     'dog_2_Ch2': dog_2_Ch2_media,
     'stim_eog0': stim_eog0,
     'stim_eog1': stim_eog1,
     'stim_eog2': stim_eog2,
     'x_start': x_start,
     'y_start': y_start,
     'x_end': x_end,
     'y_end': y_end})

linear_regression_df.drop(linear_regression_df.tail(1).index,inplace=True)
linear_regression_df.head()


# Linear Regression

The values of channel 0,1 and 2 at DoG peaks 1 and 2 are used as features for the linear regression.
The regression targets are the x and y coordinates at stimulus time.

In [ ]:
#Regression with second Dog peaks, start coordinates and stimulus
%matplotlib notebook
#Split data frame in data and targets
regression_data = linear_regression_df.loc[:,['dog_2_Ch0','dog_2_Ch1','dog_2_Ch2','dog_1_Ch0','dog_1_Ch1','dog_1_Ch2']]

regression_targets = linear_regression_df.loc[:, ['x_end','y_end']]
                        
# Split the data into training/testing sets
eog_ch_train = regression_data[:210]
eog_ch_test = regression_data[210:]

# Split the targets into training/testing sets
eog_coor_train = regression_targets[:210]
eog_coor_test = regression_targets[210:]

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(eog_ch_train, eog_coor_train)
# Make predictions using the testing set
eog_coor_pred = regr.predict(eog_ch_test)

# The coefficients
print('Regression with second Dog peaks, start coordinates and stimulus')
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(eog_coor_test, eog_coor_pred))
print("Distance in pixels: %.2f " % np.sqrt(mean_squared_error(eog_coor_test, eog_coor_pred)))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(eog_coor_test, eog_coor_pred))

# Plot outputs
n = eog_coor_test.shape[0]
fig3 = plt.figure(figsize=(14, 8))
    # create the function that will do the plotting, where curr is the current frame
def update(curr):
     
    # check if animation is at the last frame, and if so, stop the animation.
    if curr == n: 
        a.event_source.stop()
    
    plt.scatter(eog_coor_pred[0:curr,0] ,eog_coor_pred[0:curr,1], color = "red")
    plt.scatter(eog_coor_test.iloc[0:curr, [0]] ,eog_coor_test.iloc[0:curr, [1]], color = 'blue')
    plt.xlim(0, 2560)
    plt.ylim(1440, 0)
    plt.show()
   
    
    
a = animation.FuncAnimation(fig3, update, interval=500)

# Classifying Gaze predictions in quadrants

The screen is divided in 9 quadrants in order to know how often the prediction falls into the same section as the true stimulus coordinates.
A possible improvement for this section could be to convert the area into Polar coordinates and predict quadrants by the direccion of gaze.

In [ ]:
#Adds quadrant label to each prediction and test point
def quadrand_labels(df):
    df['quadrant label'] = 0   
    for i, row in enumerate(df.iterrows()):
            x = df.loc[i,'x']
            y = df.loc[i,'y']

            if x >=0 and x <854:
                if y >=960 and y <= 1440:
                    df.loc[i,'quadrant label'] = 1
                elif y >=480 and y < 960:
                    df.loc[i,'quadrant label'] = 4
                elif y >=0 and y < 480:
                    df.loc[i,'quadrant label'] = 7
            elif x >=854 and x <1708:
                if y >=960 and y <= 1440:
                    df.loc[i,'quadrant label'] = 2
                elif y >=480 and y < 960:
                    df.loc[i,'quadrant label'] = 5
                elif y >=0 and y < 480:
                    df.loc[i,'quadrant label'] = 8
            elif x >=1708 and x <=2562:
                if y >=960 and y <= 1440:
                    df.loc[i,'quadrant label'] = 3
                elif y >=480 and y < 960:
                    df.loc[i,'quadrant label'] = 6
                elif y >=0 and y < 480:
                    df.loc[i,'quadrant label'] = 9

    return df

eog_coor_pred_df = pd.DataFrame(eog_coor_pred,columns=['x','y'] )

eog_coor_test.columns = ['x','y'] 
eog_coor_test.reset_index(inplace = True, drop = True)
labeled_predict = quadrand_labels(eog_coor_pred_df)
labeled_test = quadrand_labels(eog_coor_test)

In [ ]:
#Creates grid for classifying data in quadrants
calibration_points = [
    (50, 50),
    (2510, 50),
    (50, 1390),
    (2510, 1390),
    (1280, 720)
]
n = len(eog_coor_test)
fig5 = plt.figure(figsize=(14, 8))
def update(curr):
   
    # Animation
    if curr == n: 
        a.event_source.stop()
 
    plt.xlim(0, 2560)
    plt.ylim(1440, 0)
    plt.gca().set_aspect('equal', 'box')
    plt.legend()
    plt.axvline(854, color='black')
    plt.axvline(1708, color='black')
    plt.axhline(480, color='black')
    plt.axhline(960, color='black')
    plt.scatter(eog_coor_pred[curr][0] ,eog_coor_pred[curr][1], color = "red")
    plt.scatter(eog_coor_test['x'].iloc[curr],eog_coor_test['y'].iloc[curr] , color = 'blue')
    plt.xlim(0, 2560)
    plt.ylim(1440, 0)
    if labeled_test['quadrant label'].iloc[curr] == labeled_predict['quadrant label'].iloc[curr]:
        plt.title('Stimulus nr {} : Match'.format(curr))
    else:
        plt.title('Stimulus nr {} : Mismatch'.format(curr))
    plt.show()
        
a = animation.FuncAnimation(fig5, update, interval=1500)

# Percentage of matches
total_match = sum(labeled_test['quadrant label'] == labeled_predict['quadrant label'])
print('Total match is {} from {}, {}% match'.format(total_match,len(eog_coor_pred_df),((total_match*100)/len(eog_coor_pred_df)).astype(int)))

# Gaze draw prediction
This part tests if the figures formed by connecting the stimulus coordinates can be replicated by the predictions.

In [ ]:
#Plot of experiment stimulus points
%matplotlib notebook
stimulus_df = accepted_datasets_df[accepted_datasets_df['stimulus'] == 1]
stim_x_no_calib = stimulus_df[stimulus_df['calibration point'] == 0]
#stimulus_time = stimulus_df['time stamp']
stimulus_x = stimulus_df['x']
stimulus_y = stimulus_df['y']
stimulus_df.reset_index(inplace= True)

fig = plt.figure(figsize= (12,6))
plt.xlim(0, 2560)
plt.ylim(1440, 0)

n = 11
def update(curr):
    
    # check if animation is at the last frame, and if so, stop the animation.
    if curr == 11: 
        a.event_source.stop()   
        
    if stimulus_df.loc[curr,'calibration point'] == 1.0:
        plt.clf()
        plt.xlim(0, 2560)
        plt.ylim(1440, 0)
        plt.title(curr)
        plt.axvline(854, color='black')
        plt.axvline(1708, color='black')
        plt.axhline(480, color='black')
        plt.axhline(960, color='black')
        plt.gca().set_aspect('equal', 'box')
        #plt.scatter(eog_coor_pred[0:curr,0] ,eog_coor_pred[0:curr,1], color = "blue")
        plt.scatter(stimulus_df.loc[curr , 'x'] ,stimulus_df.loc[curr , 'y'])#, markersize=20)
        
        plt.show()
        
    else:
        plt.clf()
        plt.xlim(0, 2560)
        plt.ylim(1440, 0)
        plt.title(curr)
        plt.axvline(854, color='black')
        plt.axvline(1708, color='black')
        plt.axhline(480, color='black')
        plt.axhline(960, color='black')
        plt.gca().set_aspect('equal', 'box') 
        plt.plot(eog_coor_pred[0:curr,0] ,eog_coor_pred[0:curr,1], color = "red")
        plt.plot(eog_coor_test.iloc[0:curr,[0]] ,eog_coor_test.iloc[0:curr, [1]], color = 'blue')
        plt.show()
       
    
gaze_draw = animation.FuncAnimation(fig, update, interval=1000)